In [1]:
from sqlalchemy import create_engine
from pandas import read_csv

In [2]:
column_names = read_csv('/scratch/ar1515/QETS_Project/archive/taq_aug_2019_quotes_500_tickers.csv', nrows=1, parse_dates=[[0, 1]],
                       infer_datetime_format=True).columns
column_names

Index(['DATE_TIME_M', 'EX', 'BID', 'BIDSIZ', 'ASK', 'ASKSIZ', 'SYM_ROOT',
       'SYM_SUFFIX'],
      dtype='object')

In [3]:
sql_engine = create_engine('sqlite:////scratch/ar1515/QETS_Project/archive/taq_aug_2019_test.db')
quotes = read_csv('/scratch/ar1515/QETS_Project/archive/taq_aug_2019_quotes_500_tickers.csv', chunksize=10**6, parse_dates=[[0, 1]],
                 infer_datetime_format=True, nrows=10**9, skiprows=2*(10**9), header=None)

In [ ]:
for chunk in quotes:
    chunk.columns = column_names
    chunk.drop('SYM_SUFFIX', inplace=True, axis=1)
    for ticker in chunk.SYM_ROOT.unique():
        chunk.to_sql('quotes_'+ticker, sql_engine, if_exists='append')

In [ ]:
sql_engine.dispose()

In [10]:
from pandas import read_sql

sql_engine = create_engine('sqlite:////scratch/ar1515/QETS_Project/archive/taq_aug_2019_test.db')
tables = read_sql('select name from sqlite_master where type=\'table\' order by name', sql_engine)
len(tables) # 108

222

In [10]:
read_sql('pragma table_info(quotes_C)', sql_engine)

,cid,name,type,notnull,dflt_value,pk
0,0,index,BIGINT,0,None,0
1,1,DATE_TIME_M,DATETIME,0,None,0
2,2,EX,TEXT,0,None,0
3,3,BID,FLOAT,0,None,0
4,4,BIDSIZ,BIGINT,0,None,0
5,5,ASK,FLOAT,0,None,0
6,6,ASKSIZ,BIGINT,0,None,0
7,7,SYM_ROOT,TEXT,0,None,0


In [18]:
read_sql('select max(index) from quotes_C', sql_engine)

OperationalError: (sqlite3.OperationalError) near "index": syntax error
[SQL: select max(index) from quotes_C]
(Background on this error at: http://sqlalche.me/e/e3q8)